In [ ]:
from typing import List, Optional
from kor.nodes import Object, Text
import re

In [ ]:
def bill_cleaner(path):

    """
    Función que devuelve el texto procesado de una factura.

    Input:
        - path(str): Ruta de la factura.pdf
    
    Output:
        - texto_limpio (str)
    
    """

    factura = PdfReader(path)
    
    texto_factura = ""
    for pagina in factura.pages:
        texto_factura += pagina.extract_text()
    
    # Elimino hiperlinks:
    texto_limpio = re.sub(r'\b(?:http://|https://|www\.)?\S+(?:-|\s)?\S*?(?:\.com|\.es)\b', "", texto_factura).strip()
    
    # Elimino conjuntos de puntos mayores a 1:
    texto_limpio = re.sub(r'\.{2,}', "", texto_limpio).strip()
    
    # Elimino espacios multiples en blanco y saltos de linea:
    texto_limpio = re.sub(r"\s+", " ", texto_limpio)

    return texto_limpio

# 1. Guardo el schema

In [ ]:
path_input = "data\\factura_800.pdf"

In [ ]:
input = bill_cleaner(path_input)

In [ ]:
schema = Object(
    id="informacion_factura",
    description="Informacion del recibo de la luz de una compañia electrica de un determinado cliente",
    attributes=[
        Text(
            id="nombre_cliente",
            description="El nombre y los apellidos del cliente",
        ),
        Text(
            id="dni_cliente",
            description="El documento de identificacion fiscal del cliente",
        ),
        Text(
            id="calle_cliente",
            description="La direccion de la calle del cliente",
        ),
        Text(
            id="cp_cliente",
            description="El codigo postal del cliente",
        ),
        Text(
            id="población_cliente",
            description="La poblacion en la que vive el cliente",
        ),
        Text(
            id="provincia_cliente",
            description="La provincia en la que vive el cliente",
        ),
        Text(
            id="nombre_comercializadora",
            description="Nombre de la comercializadora electrica",
        ),
        Text(
            id="cif_comercializadora",
            description="El codigo de identificacion fiscal de la comercializadora electrica",
        ),
        Text(
            id="dirección_comercializadora",
            description="La direccion de la comercializadora electrica",
        ),
        Text(
            id="cp_comercializadora",
            description="El codigo postal de la comercializadora electrica",
        ),
        Text(
            id="población_comercializadora",
            description="La poblacion de la comercializadora electrica",
        ),
        Text(
            id="provincia_comercializadora",
            description="La provincia de la comercializadora electrica",
        ),
        Text(
            id="número_factura",
            description="El numero asociado a la factura",
        ),
        Text(
            id="inicio_periodo",
            description="El inicio del periodo de consumo",
        ),
        Text(
            id="fin_periodo",
            description="El fin del periodo de consumo",
        ),
        Text(
            id="importe_factura",
            description="El importe total de la factura electrica, utilizando ',' para separar los decimales",
        ),
        Text(
            id="fecha_cargo",
            description="La fecha de cobro del importe de la factura electrica",
        ),
        Text(
            id="consumo_periodo",
            description="El consumo del periodo, utilizando ',' para separar los decimales",
        ),
        Text(
            id="potencia_contratada",
            description="La potencia contratada por el cliente, utilizando ',' para separar los decimales",
        ),

    ],
    examples=[
        (
            input,
            [
                {'nombre_cliente': 'JERÓNIMO URIARTE IZQUIERDO', 'dni_cliente': '94985339P', 'calle_cliente': 'Camino de Valdemanco', 'cp_cliente': '10183', 'población_cliente': 'Torrequemada', 'provincia_cliente': 'Cáceres', 'nombre_comercializadora': 'SECOM CENTRAL DE COMPRAS SOCIEDAD LIMITADA', 'cif_comercializadora': 'B40605925', 'dirección_comercializadora': 'PEDRO ITURRALDE OCHOA 11', 'cp_comercializadora': '46900', 'población_comercializadora': 'TORRENT', 'provincia_comercializadora': 'VALENCIA', 'número_factura': 'RQ4694566687', 'inicio_periodo': '27.10.2014', 'fin_periodo': '26.12.2014', 'importe_factura': '253,54', 'fecha_cargo': '31.12.2014', 'consumo_periodo': '796', 'potencia_contratada': '3,310'}
            ],
        )
    ],
    many=False,
)

In [74]:
with open('utils/schema.pkl', 'wb') as f:
    pickle.dump(schema, f)

# 2. Prueba con Llama3-8b local.

Sin buenos resultados

In [ ]:
llm = ChatOpenAI(model="llama3",
                 base_url="http://localhost:11434/v1",
                 openai_api_key= api_key)

# 3. Preparacion datos para finetuning Llama3-8b

In [ ]:
data_path = "data\\factura_"

In [ ]:
info_finetuning = []


for i in tqdm.tqdm(range(1000)):
    # Output
    with open(data_path + f"{i}.json", 'r', encoding='utf-8') as archivo:
        factura_i = json.load(archivo)

    # Input
    factura = PdfReader(data_path + f"{i}.pdf")
    
    texto_factura = ""
    for page in factura.pages:
        texto_factura += page.extract_text()
        
    # Elimino saltos de linea
    texto_limpio = re.sub(r"\s+", " ", texto_factura).strip()    
    # Elimino puntos
    texto_limpio = re.sub(r"\.+", "", texto_limpio)    
    # Elimino espacios multiples
    texto_limpio = re.sub(r"\s+", " ", texto_limpio)

    output = factura_i
    input = texto_limpio
    instruction = "Extrae la siguiente informacion en formato JSON de la factura electrica proporcionada: nombre del cliente, dni del cliente, calle del cliente, codigo postal del cliente, poblacion del cliente, provincia del cliente, nombre de la comercializadora, codigo de identificacion fiscal de la comercializadora, direccion de la comercializadora, codigo postal de la comercializadora, la poblacion de la comercializadora, la provincia de la comercializadora, el numero de factura, el inicio del periodo de consumo, el fin del periodo de consumo, el importe total de la factura, la fecha de cargo, el consumo del periodo y la potencia contratada."

    info_finetuning.append([output, input, instruction])

df_finetuning = pd.DataFrame(info_finetuning, columns= ["output", "input", "instruction"])
df_finetuning.to_csv("df_finetuning.csv", sep= ",", index= False)

In [96]:
df_finetuning

,output,input,instruction
0,"{'nombre_cliente': 'Conrado Daniel Iglesias', ...",DATOS DE LA FACTURA Nº factura: SV5043664894 R...,Extrae la siguiente informacion en formato JSO...
1,"{'nombre_cliente': 'LEONARDA JARAMILLO BÁEZ', ...","Lunes a sábado, de 8 a 22 horas Contratación P...",Extrae la siguiente informacion en formato JSO...
2,{'nombre_cliente': 'BENEDICTA GALLEGOS AGUILAR...,DATOS DE LA FACTURA Nº factura: H4623704265 Re...,Extrae la siguiente informacion en formato JSO...
3,"{'nombre_cliente': 'Belinda Zetina Mijares', '...",DATOS DE LA FACTURA Nº factura: SF3956122542 R...,Extrae la siguiente informacion en formato JSO...
4,{'nombre_cliente': 'PANTALEÓN VELASCO DE ALBA'...,"DATOS DE LA FACTURA IMPORTE FACTURA: 61,84 € N...",Extrae la siguiente informacion en formato JSO...
...,...,...,...
995,"{'nombre_cliente': 'SULPICIO ESCOVAR FONSECA',...",Página 1 / 2 ELECTRICA NTRA SRA DE GRACIA SDAD...,Extrae la siguiente informacion en formato JSO...
996,"{'nombre_cliente': 'Petrona Uribe Naranjo', 'd...",DATOS DE LA FACTURA Nº factura: U2093855017 Re...,Extrae la siguiente informacion en formato JSO...
997,{'nombre_cliente': 'CELESTINA TREMINIO VALLEJO...,"DATOS DE LA FACTURA IMPORTE FACTURA: 29,30 € N...",Extrae la siguiente informacion en formato JSO...
998,"{'nombre_cliente': 'Dela Anaya Naranjo', 'dni_...",Página 1 / 2 ENERGÉTICA DEL ESTE SL CIF B40563...,Extrae la siguiente informacion en formato JSO...


In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.26" trl peft accelerate bitsandbytes

In [ ]:
from unsloth import FastLanguageModel
import torch

In [ ]:
max_seq_length = 8192 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


==((====))==  Unsloth: Fast Llama patching release 2024.4
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.25.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
!pip install galore_torch

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


## Preparo el prompt

In [3]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [4]:
hf_token = os.getenv("HF_TOKEN")

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [ ]:
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

In [ ]:
train = df_finetuning[:800]
validation = df_finetuning[800:]

In [ ]:
from datasets import Dataset

dataset_finetuning = Dataset.from_pandas(train)

In [ ]:
dataset_finetuning = dataset_finetuning.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from galore_torch import GaLoreAdamW8bit
import torch.nn as nn
galore_params = []
target_modules_list = ["attn", "mlp"]
for module_name, module in model.named_modules():
    if not isinstance(module, nn.Linear):
        continue

    if not any(target_key in module_name for target_key in target_modules_list):
        continue

    print('mod ', module_name)
    galore_params.append(module.weight)
id_galore_params = [id(p) for p in galore_params]
regular_params = [p for p in model.parameters() if id(p) not in id_galore_params]


param_groups = [{'params': regular_params},
                {'params': galore_params, 'rank': 64, 'update_proj_gap': 200, 'scale': 0.25, 'proj_type': 'std'}]
optimizer = GaLoreAdamW8bit(param_groups, lr=2e-5)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset_finetuning,
    optimizers=(optimizer, None),
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = True, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

mod  base_model.model.model.layers.0.self_attn.q_proj.base_layer
mod  base_model.model.model.layers.0.self_attn.q_proj.lora_A.default
mod  base_model.model.model.layers.0.self_attn.q_proj.lora_B.default
mod  base_model.model.model.layers.0.self_attn.k_proj.base_layer
mod  base_model.model.model.layers.0.self_attn.k_proj.lora_A.default
mod  base_model.model.model.layers.0.self_attn.k_proj.lora_B.default
mod  base_model.model.model.layers.0.self_attn.v_proj.base_layer
mod  base_model.model.model.layers.0.self_attn.v_proj.lora_A.default
mod  base_model.model.model.layers.0.self_attn.v_proj.lora_B.default
mod  base_model.model.model.layers.0.self_attn.o_proj.base_layer
mod  base_model.model.model.layers.0.self_attn.o_proj.lora_A.default
mod  base_model.model.model.layers.0.self_attn.o_proj.lora_B.default
mod  base_model.model.model.layers.0.mlp.gate_proj.base_layer
mod  base_model.model.model.layers.0.mlp.gate_proj.lora_A.default
mod  base_model.model.model.layers.0.mlp.gate_proj.lora_B.de

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 233 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 4
\        /    Total batch size = 4 | Total steps = 58
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,1.228700
2,1.345800
3,1.466300
4,1.376900
5,1.317700
6,1.424300
7,1.247400
8,1.400600
9,1.431900
10,1.589200


Step,Training Loss
1,1.228700
2,1.345800
3,1.466300
4,1.376900
5,1.317700
6,1.424300
7,1.247400
8,1.400600
9,1.431900
10,1.589200


In [ ]:
model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)

In [ ]:
model.push_to_hub_merged("UrkoRR/finetuned-llama-3-8B", tokenizer, save_method = "merged_16bit", token = hf_token)

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 4.57 out of 12.67 RAM for saving.


100%|██████████| 32/32 [03:21<00:00,  6.31s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Unsloth: Saving finetuned-llama-3-8B/pytorch_model-00001-of-00004.bin...
Unsloth: Saving finetuned-llama-3-8B/pytorch_model-00002-of-00004.bin...
Unsloth: Saving finetuned-llama-3-8B/pytorch_model-00003-of-00004.bin...
Unsloth: Saving finetuned-llama-3-8B/pytorch_model-00004-of-00004.bin...


README.md:   0%|          | 0.00/573 [00:00<?, ?B/s]

pytorch_model-00002-of-00004.bin:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

pytorch_model-00001-of-00004.bin:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

pytorch_model-00003-of-00004.bin:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

pytorch_model-00004-of-00004.bin:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Done.
Saved merged model to https://huggingface.co/UrkoRR/finetuned-llama-3-8B


### Prueba:

In [ ]:
instruction = "Extrae la siguiente informacion en formato JSON de la factura electrica proporcionada: nombre del cliente, dni del cliente, calle del cliente, codigo postal del cliente, poblacion del cliente, provincia del cliente, nombre de la comercializadora, codigo de identificacion fiscal de la comercializadora, direccion de la comercializadora, codigo postal de la comercializadora, la poblacion de la comercializadora, la provincia de la comercializadora, el numero de factura, el inicio del periodo de consumo, el fin del periodo de consumo, el importe total de la factura, la fecha de cargo, el consumo del periodo y la potencia contratada."
input = validation.loc[800].input

In [ ]:
FastLanguageModel.for_inference(model)
inputs = tokenizer([alpaca_prompt.format(instruction, # instruction
                                         input, # input
                                         "", # output
                                        )],
                   return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)